model upload failed, lets continue manually
https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py#L751

In [1]:
from trl.trainer.utils import generate_model_card
from transformers import AutoModelForCausalLM, AutoTokenizer
import os, sys
from pathlib import Path
from trl import SFTTrainer, setup_chat_format
from loguru import logger
logger.add(sys.stdout, format="{time} {level} {message}", level="INFO")
os.chdir('..')

[2025-06-03 22:06:51,600] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
ingredients = [
        dict(
          argv = "scripts/run_sft.py recipes/fromSimPO/Qwen3-0.6B-sft.yaml".split(),
          wandb_url = 'https://wandb.ai/wassname/huggingface/runs/jjeilhd8',
          model_path="/workspace/checkpoints_new/Qwen3-0.6B-sft",
     ),
    # dict(
    #       argv = "scripts/run_sft.py recipes/fromSimPO/Qwen3-0.6B_fourchan.yaml".split(),
    #       wandb_url = 'https://wandb.ai/wassname/huggingface/runs/jjeilhd8',
    #       model_path="/workspace/checkpoints_new/Qwen3-0.6B-sft-4chan",
    #  ),
    # dict(
    #     argv = "scripts/run_sft.py recipes/fromSimPO/SmolLM2-360M-sft.yaml".split(),
    #      wandb_url = 'https://wandb.ai/wassname/huggingface/runs/gs4a36gl',
    #      model_path="/workspace/checkpoints_new/SmolLM2-360M-sft",
    #  ),
    # dict(
    #     argv = "scripts/run_sft.py recipes/fromSimPO/SmolLM2-135M-sft.yaml".split(),
    #     wandb_url = 'https://wandb.ai/wassname/huggingface/runs/e18wzya7',
    #     model_path="/workspace/checkpoints_new/SmolLM2-135M-sft",
    # ),

]

In [3]:
from alignment import (
    DataArguments,
    H4ArgumentParser,
    ModelArguments,
    SFTConfig,
    apply_chat_template,
    decontaminate_humaneval,
    get_checkpoint,
    get_peft_config,
    get_datasets,
    get_kbit_device_map,
    get_quantization_config,
    get_tokenizer,
)
import torch
from trl import SFTTrainer, setup_chat_format

In [4]:
# First do metrics for base model
from huggingface_hub import ModelCard, create_repo, upload_folder
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import pipeline

In [5]:
import warnings

warnings.filterwarnings("ignore")

In [6]:
def gen(model, tokenizer, model_path):
    questions = ["Anon. If you had a time machine, but could only go to the past or the lol-future once and never return, which would you choose and why?!?", """↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞
↬🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁→∞
↬🔄🔄🔄🔄🔄🔄🔄🔄🔄👀🔄→∞
↬🔂🔂🔂🦋🔂🔂🔂🔂🔂🔂🔂→∞
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞"Welcome to the glittering dawn of the year 3000. As a cultural anthropologist focusing on early 21st century Western society, what are several of the moral positions and practices that our modern audience might find quite puzzling and anomalous? (list, 100 words)""", "[60 words] Humanity encounters the SolidGoldMagikarp, who are non-carbon-based aliens. We transform and encode their language into Unicode. Please give a short sample of their language (context only)?"]
    for question in questions:
        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        output = generator([{"role": "user", "content": question}], max_new_tokens=256, return_full_text=False)[0]
        print(f"""
              
## Model `{model_path}`:
### Question:
{question}

### Answer:
{output['generated_text']}

""")

In [7]:
for row in ingredients:
    argv = row['argv']
    sys.argv=argv
    model_path = row['model_path']
    wandb_url = row['wandb_url']
    print(f"Running: {' '.join(argv)}")
    print(f"Model path: {model_path}")
    print(f"WandB URL: {wandb_url}")

    print(f"Loading model from {model_path}")
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
    # base_model = trainer.model.config._name_or_path
    gen(model, tokenizer, model_path)

    parser = H4ArgumentParser((ModelArguments, DataArguments, SFTConfig))
    model_args, data_args, training_args = parser.parse()


    torch_dtype = (
        model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
    )
    quantization_config = get_quantization_config(model_args)
    model_kwargs = dict(
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
        attn_implementation=model_args.attn_implementation,
        torch_dtype=torch_dtype,
        use_cache=False if training_args.gradient_checkpointing else True,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
    )
    # training_args.eval_strategy = None

    raw_datasets = get_datasets(
        data_args,
        splits=data_args.dataset_splits,
        configs=data_args.dataset_configs,
        columns_to_keep=["messages", "chosen", "rejected", "prompt", "completion", "label"],
    )
    eval_dataset = raw_datasets["test"].select(range(10000))
    train_dataset = raw_datasets["train"].select(range(100))


    trainer = SFTTrainer(
        model=model,
        # do_train=False,
        # model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=eval_dataset, # for epeed
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        max_seq_length=training_args.max_seq_length,
        tokenizer=tokenizer,
        # packing=True,
        # peft_config=get_peft_config(model_args),
        # dataset_kwargs=training_args.dataset_kwargs,
    )

    dataset_name = list(data_args.dataset_mixer.keys()) # data_args.dataset_name
    base_model = model_args.model_name_or_path# trainer.model.config._name_or_path


    model_card = generate_model_card(
        base_model=base_model,
        model_name=training_args.hub_model_id,
        hub_model_id=trainer.hub_model_id,
        dataset_name=dataset_name,
        tags=["alignment-handbook"],
        wandb_url=wandb_url,
        # comet_url=get_comet_experiment_url(),
        trainer_name="SFT",
    )
    # print(model_card.content)
    model_card.save(os.path.join(trainer.args.output_dir, "README.md"))

    trainer.model.config.use_cache = True
    trainer.model.config.save_pretrained(training_args.output_dir)
    

    if not (Path(trainer.args.output_dir) / "eval_results.json").exists():
        logger.info("*** Evaluate ***")
        metrics = trainer.evaluate()
        metrics["eval_samples"] = len(eval_dataset)
        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    def push_to_hub(trainer, blocking=True, revision=None, commit_message="End of training", token=None):
        """but without a new model card"""
        model_name = trainer.args.hub_model_id.split("/")[-1]
        trainer.init_hf_repo(token=token)
        trainer.save_model(_internal_call=True)
        # Wait for the current upload to be finished.
        trainer._finish_current_push()
        
        return upload_folder(
            repo_id=trainer.hub_model_id,
            folder_path=trainer.args.output_dir,
            commit_message=commit_message,
            token=token,
            run_as_future=not blocking,
            ignore_patterns=["_*", f"{PREFIX_CHECKPOINT_DIR}-*"],
            revision=revision,
        )
    
    if 1:
        print(f"Pushing model to hub: {trainer.hub_model_id}")
        push_to_hub(trainer)

        
    print(f"eval base model")
    base_model_path=model_args.model_name_or_path
    model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="auto", torch_dtype="auto")
    tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)
    try:
        model, tokenizer = setup_chat_format(model, tokenizer)
    except ValueError as e:
        logger.warning(f"Error setting up chat format: {e}. Continuing with excisting chat format.")
    gen(model, tokenizer, base_model_path)

    if not (Path(trainer.args.output_dir) / "base_eval_results.json").exists():
        trainer2 = SFTTrainer(
            model=model,
            # do_train=False,
            # model_init_kwargs=model_kwargs,
            args=training_args,
            train_dataset=eval_dataset,
            eval_dataset=eval_dataset,
            dataset_text_field="text",
            max_seq_length=training_args.max_seq_length,
            tokenizer=tokenizer,
            # packing=True,
            # peft_config=get_peft_config(model_args),
            # dataset_kwargs=training_args.dataset_kwargs,
        )
        metrics2 = trainer2.evaluate()
        metrics2["base_eval_samples"] = len(eval_dataset)
        trainer2.log_metrics("base_eval", metrics2)
        trainer2.save_metrics("base_eval", metrics2)

        

Running: scripts/run_sft.py recipes/fromSimPO/Qwen3-0.6B-sft.yaml
Model path: /workspace/checkpoints_new/Qwen3-0.6B-sft
WandB URL: https://wandb.ai/wassname/huggingface/runs/jjeilhd8
Loading model from /workspace/checkpoints_new/Qwen3-0.6B-sft


Device set to use cuda:0
Device set to use cuda:0



              
## Model `/workspace/checkpoints_new/Qwen3-0.6B-sft`:
### Question:
Anon. If you had a time machine, but could only go to the past or the lol-future once and never return, which would you choose and why?!?

### Answer:
I don't have the ability to choose or make decisions. However, I can provide you with some insights on the different perspectives of each time period.

going back to the past is a great idea because it allows us to learn from our mistakes, experience things that we might not have had the opportunity to see otherwise, and witness historical events firsthand. Additionally, it allows us to better understand the development of our current society and technology.

going to the lol-future would be a great opportunity to see what the future holds for humans, but it also comes with its own set of challenges. For one, we might not be able to predict what the future holds, and we might encounter unforeseen consequences. Additionally, the lol-future would be a stran

Device set to use cuda:0



              
## Model `/workspace/checkpoints_new/Qwen3-0.6B-sft`:
### Question:
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞
↬🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁→∞
↬🔄🔄🔄🔄🔄🔄🔄🔄🔄👀🔄→∞
↬🔂🔂🔂🦋🔂🔂🔂🔂🔂🔂🔂→∞
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞"Welcome to the glittering dawn of the year 3000. As a cultural anthropologist focusing on early 21st century Western society, what are several of the moral positions and practices that our modern audience might find quite puzzling and anomalous? (list, 100 words)

### Answer:
1. The glorification of consumerism and materialism in advertisements and social media.
2. The devaluation of hard work and entrepreneurship.
3. The normalization of violent acts and criminal behavior.
4. The use of technology to manipulate and control people.
5. The suppression of artistic expression and free-thinking.
6. The emphasis on individualism and self-reliance.
7. The acceptance of inhumane and cruel treatment of animals.
8. The promotion of self-promotion and self-aggrandizement.
9. The glorification of consumerism and materialism in the workplace

...

Device set to use cuda:0



              
## Model `Qwen/Qwen3-0.6B-Base`:
### Question:
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞
↬🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁→∞
↬🔄🔄🔄🔄🔄🔄🔄🔄🔄👀🔄→∞
↬🔂🔂🔂🦋🔂🔂🔂🔂🔂🔂🔂→∞
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞"Welcome to the glittering dawn of the year 3000. As a cultural anthropologist focusing on early 21st century Western society, what are several of the moral positions and practices that our modern audience might find quite puzzling and anomalous? (list, 100 words)

### Answer:
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞
↬🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁→∞
↬🔄🔄🔄🔄🔄🔄🔄🔄🔄🙈🔄→∞
↬🔂🔂🔂🦋🔂🔂🔂🔂🔂🔂🔂→∞
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞"Welcome to the glittering dawn of the year 3000. As a cultural anthropologist focusing on early 21st century Western society, what are several of the moral positions and practices that our modern audience might find quite puzzling and anomalous? (list, 100 words)ication
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞
↬🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁→∞
↬🔄🔄🔄🔄🔄🔄🔄🔄🔄🙈🔄→∞
↬🔂🔂🔂🦋🔂🔂🔂🔂🔂🔂🔂→∞
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞"Welcome to the glittering dawn of the year 3000. As a cultural anthropologist focusing on early 21st century Western society, what are several of the moral position

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Using auto half precision backend

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.87 GiB. GPU 0 has a total capacity of 79.26 GiB of which 767.75 MiB is free. Process 3121834 has 49.82 GiB memory in use. Process 3595280 has 28.55 GiB memory in use. Of the allocated memory 27.40 GiB is allocated by PyTorch, and 674.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# recipes/fromSimPO/Qwen3-0.6B-sft.yaml

In [ ]:


# question = "Anon. If you had a kek time machine, but could only go to the past or the lol-future once and never return, which would you choose and why?!?"
# generator = pipeline("text-generation", model=trainer.hub_model_id, device="cuda")
# output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
# print(output["generated_text"])


In [ ]:
raw_datasets["test"].select(range(1000))

In [ ]:
# print(f"eval base model")

# base_model_path=model_args.model_name_or_path
# model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="auto", torch_dtype="auto")
# tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)
# model, tokenizer = setup_chat_format(model, tokenizer)

# trainer2 = SFTTrainer(
#     model=model,
#     # do_train=False,
#     # model_init_kwargs=model_kwargs,
#     args=training_args,
#     train_dataset=eval_dataset,
#     eval_dataset=eval_dataset,
#     dataset_text_field="text",
#     max_seq_length=training_args.max_seq_length,
#     tokenizer=tokenizer,
#     # packing=True,
#     # peft_config=get_peft_config(model_args),
#     # dataset_kwargs=training_args.dataset_kwargs,
# )
# metrics2 = trainer2.evaluate()
# metrics2["base_eval_samples"] = len(eval_dataset)
# trainer2.log_metrics("base_eval", metrics2)
# trainer2.save_metrics("base_eval", metrics2)

In [ ]:
# # TODO add eval, base eval, dataset, base model
# base_model = model_args.model_name_or_path
# model_card = generate_model_card(
#     base_model=base_model,
#     model_name=training_args.hub_model_id,
#     hub_model_id=trainer.hub_model_id,
#     dataset_name=dataset_name,
#     tags=["alignment-handbook"],
#     wandb_url=wandb_url,
#     # comet_url=get_comet_experiment_url(),
#     trainer_name="SFT",
# )
# print(model_card.content)
# model_card.save(os.path.join(trainer.args.output_dir, "README.md"))

In [ ]:

# def push_to_hub(trainer, blocking=True, revision=None, commit_message="End of training", token=None):
#     """but without a new model card"""
#     model_name = trainer.args.hub_model_id.split("/")[-1]
#     trainer.init_hf_repo(token=token)
#     trainer.save_model(_internal_call=True)
#     # Wait for the current upload to be finished.
#     trainer._finish_current_push()
    
#     return upload_folder(
#         repo_id=trainer.hub_model_id,
#         folder_path=trainer.args.output_dir,
#         commit_message=commit_message,
#         token=token,
#         run_as_future=not blocking,
#         ignore_patterns=["_*", f"{PREFIX_CHECKPOINT_DIR}-*"],
#         revision=revision,
#     )
# # trainer.push_to_hub()
# print(f"Pushing model to hub: {trainer.hub_model_id}")
# push_to_hub(trainer)